In [25]:
import os
import jieba
import string

### 1. Load Text Data

In [45]:
with open('ham_5000.utf8', encoding='utf-8') as f:
    ham_docs = f.readlines()
with open('spam_5001.utf8', encoding='utf-8') as f:
    spam_docs = f.readlines()

### 2. Clean Text Data
#### 2.1 Split into Tokens
- We can filter out punctuation from tokens.
- We can remove tokens that are just punctuation or contain numbers by using an isalpha() check on each token.
- We can remove stop words.

In [55]:
with open('stopwords.txt', encoding='utf-8') as f:
    stopwords = f.read().split('\n')
    
def cleanText(docs):
    cleanedText = []
    for doc in docs:
        words = jieba.cut(doc)
        cleanedWords = [word for word in words
                       if word.isalpha() and word not in stopwords]
        sentence = ' '.join(cleanedWords)
        cleanedText.append(sentence)
    return cleanedText

clean_ham = cleanText(ham_docs)

from collections import Counter
cnt = Counter(' '.join(clean_ham))
cnt

Counter({'讲': 276,
         ' ': 243635,
         '孔': 27,
         '子': 2842,
         '后': 742,
         '人': 7394,
         '故': 225,
         '事': 1945,
         '一': 7312,
         '个': 4727,
         '老': 1618,
         '领': 171,
         '导': 350,
         '回': 1179,
         '到': 1614,
         '家': 1806,
         '乡': 41,
         '儿': 446,
         '感': 1792,
         '情': 2321,
         '不': 9828,
         '贪': 10,
         '财': 62,
         '孙': 27,
         '为': 507,
         '和': 114,
         '睦': 9,
         '弟': 230,
         '魏': 6,
         '宗': 27,
         '万': 232,
         '马': 292,
         '车': 475,
         '有': 1882,
         '洋': 40,
         '妞': 15,
         '大': 2955,
         '概': 176,
         '考': 486,
         '察': 55,
         '民': 143,
         '俗': 98,
         '过': 1144,
         '年': 1902,
         '总': 404,
         '想': 3111,
         '出': 1380,
         '国': 1231,
         '爷': 78,
         '教': 614,
         '育': 206,
         '解': 562,
     

In [ ]:
cnt